# Train with RAPIDS

description: train with RAPIDS and cuML on a subset of the airlines dataset

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# training script
script_dir = prefix.joinpath("code", "models", "rapids")
script_name = "train.py"

# environment file
environment_file = prefix.joinpath("environments", "rapids-example.dockerfile")

# azure ml settings
environment_name = "rapids-airline-example"
experiment_name = "rapids-airline-example"
compute_target = "gpu-V100-1"

In [ ]:
print(open(script_dir.joinpath(script_name)).read())

In [ ]:
from azureml.core import ScriptRunConfig, Experiment, Environment, Dataset

ds = Dataset.File.from_files(
    "https://airlinedataset.blob.core.windows.net/airline-20m/*"
)

arguments = [
    "--data_dir",
    ds.as_mount(),
    "--n_bins",
    32,
    "--compute",
    "single-GPU",
    "--cv-folds",
    1,
]

env = Environment(environment_name)
env.docker.enabled = True
env.docker.base_image = None
env.docker.base_dockerfile = environment_file
env.python.user_managed_dependencies = True

src = ScriptRunConfig(
    source_directory=script_dir,
    script=script_name,
    arguments=arguments,
    environment=env,
    compute_target=compute_target,
)

run = Experiment(ws, experiment_name).submit(src)
run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)